In [1]:
from mmseg.apis import init_segmentor, inference_segmentor, show_result_pyplot
from mmseg.core.evaluation import get_palette
import mmcv
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import cv2
import torch
import torch.nn.functional as F
import os

In [2]:
config_file = './mmsegmentation_configs/deeplabv3plus/deeplabv3plus_r101-d8_512x512_80k_ade20k.py'
checkpoint_file = './checkpoints/deeplabv3plus_r101-d8_512x512_80k_ade20k_20200615_014139-d5730af7.pth'
model = init_segmentor(config_file, checkpoint_file, device='cuda:2')

load checkpoint from local path: ./checkpoints/deeplabv3plus_r101-d8_512x512_80k_ade20k_20200615_014139-d5730af7.pth


In [5]:
config_file_ = './mmsegmentation_configs/deeplabv3/deeplabv3_r50-d8_512x1024_40k_cityscapes.py'
checkpoint_file_ = './checkpoints/deeplabv3_r50-d8_512x1024_40k_cityscapes_20200605_022449-acadc2f8.pth'
model_ = init_segmentor(config_file_, checkpoint_file_, device='cuda:3')

load checkpoint from local path: ./checkpoints/deeplabv3_r50-d8_512x1024_40k_cityscapes_20200605_022449-acadc2f8.pth


In [6]:
ground    = np.array([6,7,14,35,47,53,95])
building  = np.array([1,11,15,16,19,26,49,52,54,85])
non_motor = np.array([128])
motor     = np.array([20,81,116])
grass     = np.array([9,10,13,67])
people    = np.array([12])
tree      = np.array([4,5,17,18])
water     = np.array([21,22,27,48,61,114,129])

In [7]:
def trans(i):
    if i in ground:
        return 0
    if i in building:
        return 1
    if i in non_motor:
        return 2
    if i in motor:
        return 3
    if i in grass:
        return 4
    if i in people:
        return 5
    if i in tree:
        return 6
    if i in water:
        return 7
    return 1

def judge(i):
    if i > 12 and i <= 17:
        return 3
    if i == 11 or i == 18 :
        return 2
    return 1

In [8]:
#base
import os
path = '/home/cv_stu_3190102787/21_cv_final_project/MMW_BLD/images/'
f = open("test_list.txt")                
name = f.readline().strip("\n")     
k=0
while name:   
    img = path + name           
    image_array=cv2.imread(img)
    
    result_ade = inference_segmentor(model, img)[0]
    result_city = inference_segmentor(model_, img)[0]

    for i in range(image_array.shape[0]):
        for j in range(image_array.shape[1]):
            label_ade = trans(result_ade[i][j])
            label_city = judge(result_city[i][j])
            if label_ade == 7: 
                if i > 600 and j < 600:
                    image_array[i][j]=[7,7,7]
                else:
                    image_array[i][j]=[1,1,1]
            else:
                image_array[i][j]=[label_ade,label_ade,label_ade]
            if label_city == 3:
                image_array[i][j] = [3,3,3]
            #if label_city == 2 and label_ade != 3:
                #image_array[i][j] = [2,2,2]

            
    path_save = 'seg/'+name[0:23]+'_gtFine_color.png'
    cv2.imwrite(path_save, image_array, [cv2.IMWRITE_PNG_COMPRESSION, 0])
    print(k)
    k+=1
    name = f.readline().strip("\n")     
f.close()


0
1
2
3
4
5
6
7
8
9


In [9]:
#water
img = '/home/cv_stu_3190102787/21_cv_final_project/MMW_BLD/images/1102_MMW_DJI_0076_00105.jpg'
name = '1102_MMW_DJI_0076_00105.jpg'
image = Image.open(img)
image_array = np.array(image)
w = 264
h = 480
times = 8
for i in range(image_array.shape[0]//w):
    for l in range(image_array.shape[1]//h):
        l = 0
        part = image_array[i*w:(i+1)*w,l*h:(l+1)*h]
        img_tmp= np.zeros([times*w,times*h,3])
        if( np.mean(part) > 60 ):
            continue
        for m in range(3):
            torch_data=torch.from_numpy(part[:,:,m])
            b = torch.full([1,1,w,h], 0)
            b[0] = torch_data
            b = b.float()
            x = F.interpolate(b, [w*times,h*times],  mode='bilinear', align_corners=True )
            img_tmp[:,:,m] = x.numpy()[0]
        cv2.imwrite('tmp.png', img_tmp*2, [cv2.IMWRITE_PNG_COMPRESSION, 0])
        img_ = 'tmp.png'

        result = inference_segmentor(model, img_)

        res = result[0]
        pe = np.where(res==21)
        if np.array(pe).size == 0:
            continue
        k = np.zeros([res.shape[0],res.shape[1],3])
        #print(i,l)
        k[pe] = 21
        a = k[::times,::times,:]
        #print(a)
        c = np.array(np.where( a == 21 ))
        #print(c)
        #print(c[0].shape[0]/3)
        path_ = 'seg/'+name[0:23]+'_gtFine_color.png'
        image = Image.open(path_)
        image_ary = np.array(image)
        for j in range(0, c[0].shape[0], 3):
            #print(c.shape[0])
            image_ary[c[0,j]+i*w, c[1,j]+l*h] = [7,7,7]

        cv2.imwrite(path_, image_ary, [cv2.IMWRITE_PNG_COMPRESSION, 0])

In [10]:
#people
path = '/home/cv_stu_3190102787/21_cv_final_project/MMW_BLD/images/'
f = open("test_list.txt")                
name = f.readline().strip("\n")     
s=0
while name:   
    img = path + name  
    image = Image.open(img)
    image_array = np.array(image)
    w = 250
    h = 470
    times = 8
    for i in range(image_array.shape[0]//w):
        for l in range(image_array.shape[1]//h):
            part = image_array[i*w:(i+1)*w,l*h:(l+1)*h]
            img_tmp= np.zeros([times*w,times*h,3])
            for m in range(3):
                torch_data=torch.from_numpy(part[:,:,m])
                b = torch.full([1,1,w,h], 0)
                b[0] = torch_data
                b = b.float()
                x = F.interpolate(b, [w*times,h*times],  mode='bilinear', align_corners=True )
                img_tmp[:,:,m] = x.numpy()[0]
            cv2.imwrite('tmp.png', img_tmp, [cv2.IMWRITE_PNG_COMPRESSION, 0])
            img_ = 'tmp.png'

            result = inference_segmentor(model, img_)

            res = result[0]
            pe = np.where(res==12)
            if np.array(pe).size == 0:
                continue
            k = np.zeros([res.shape[0],res.shape[1],3])
            #print(i,l)
            k[pe] = 12
            a = k[::times,::times,:]
            #print(a)
            c = np.array(np.where( a == 12 ))
            #print(c)
            #print(c[0].shape[0]/3)
            if c[0].shape[0]/3 < 150 or c[0].shape[0]/3 > 560:
                continue
            path_ = 'seg/'+name[0:23]+'_gtFine_color.png'
            image = Image.open(path_)
            image_ary = np.array(image)
            for j in range(0, c[0].shape[0], 3):
                #print(c.shape[0])
                image_ary[c[0,j]+i*w, c[1,j]+l*h] = [5,5,5]

            cv2.imwrite(path_, image_ary, [cv2.IMWRITE_PNG_COMPRESSION, 0])
    print(s)
    s+=1
    name = f.readline().strip("\n")     
f.close()

0
1
2
3
4
5
6
7
8
9


In [11]:
#bike
import os
path = '/home/cv_stu_3190102787/21_cv_final_project/MMW_BLD/images/'
f = open("test_list.txt")                
name = f.readline().strip("\n")     
k=0
while name:   
    img = path + name           
    image_array=cv2.imread(img)

    result_city = inference_segmentor(model_, img)[0]
        
    path_save = 'seg/'+name[0:23]+'_gtFine_color.png'
    gt=cv2.imread(path_save)
    
    for i in range(image_array.shape[0]):
        for j in range(image_array.shape[1]):
            label_city = judge(result_city[i][j])
            if label_city == 2 and gt[i][j][0] != 3 and gt[i][j][0] != 1:
                gt[i][j] = [2,2,2]
                
            
    cv2.imwrite(path_save, gt, [cv2.IMWRITE_PNG_COMPRESSION, 0])
    print(k)
    k+=1
    name = f.readline().strip("\n")     
f.close()

0
1
2
3
4
5
6
7
8
9


In [6]:
image_name = [
    "1102_MMW_DJI_0076_00001.jpg",
    "1102_MMW_DJI_0076_00005.jpg",
    "1102_MMW_DJI_0076_00009.jpg",
    "1102_MMW_DJI_0076_00013.jpg",
    "1102_MMW_DJI_0076_00017.jpg",
    
    "1102_MMW_DJI_0076_00021.jpg",
    "1102_MMW_DJI_0076_00025.jpg",
    "1102_MMW_DJI_0076_00029.jpg",
    "1102_MMW_DJI_0076_00033.jpg",
    "1102_MMW_DJI_0076_00037.jpg",
    
    "1102_MMW_DJI_0076_00041.jpg",
    "1102_MMW_DJI_0076_00045.jpg",
    "1102_MMW_DJI_0076_00049.jpg",
    "1102_MMW_DJI_0076_00053.jpg",
    "1102_MMW_DJI_0076_00057.jpg",
    
    "1102_MMW_DJI_0076_00065.jpg",
    "1102_MMW_DJI_0076_00073.jpg",
    "1102_MMW_DJI_0076_00081.jpg",
    "1102_MMW_DJI_0076_00089.jpg",
    "1102_MMW_DJI_0076_00097.jpg",
    
    "1102_MMW_DJI_0076_00105.jpg",
    "1102_MMW_DJI_0076_00113.jpg",
    "1102_MMW_DJI_0076_00121.jpg",
    "1102_MMW_DJI_0076_00129.jpg",
    "1102_MMW_DJI_0076_00137.jpg",
    
    "1102_MMW_DJI_0076_00145.jpg",
    "1102_MMW_DJI_0076_00153.jpg",
    "1102_MMW_DJI_0076_00161.jpg",
    "1102_MMW_DJI_0076_00169.jpg",
    "1102_MMW_DJI_0076_00177.jpg",
    
    "1102_MMW_DJI_0076_00185.jpg",
    "1102_MMW_DJI_0076_00193.jpg",
    "1102_MMW_DJI_0076_00201.jpg",
    "1102_MMW_DJI_0076_00209.jpg",
    "1102_MMW_DJI_0076_00217.jpg",
    
    "1102_MMW_DJI_0076_00225.jpg",
    "1102_MMW_DJI_0076_00233.jpg",
    "1102_MMW_DJI_0076_00241.jpg",
    "1102_MMW_DJI_0076_00249.jpg",
    "1102_MMW_DJI_0076_00257.jpg",
    
    "1102_MMW_DJI_0076_00265.jpg",
    "1102_MMW_DJI_0076_00273.jpg",
    "1102_MMW_DJI_0076_00281.jpg",
    "1102_MMW_DJI_0076_00289.jpg",
    "1102_MMW_DJI_0076_00297.jpg",
    
    "1102_MMW_DJI_0076_00305.jpg",
    "1102_MMW_DJI_0076_00313.jpg",
    "1102_MMW_DJI_0076_00321.jpg",
    "1102_MMW_DJI_0076_00329.jpg",
    "1102_MMW_DJI_0076_00337.jpg",
    
    "1102_MMW_DJI_0076_00345.jpg",
    "1102_MMW_DJI_0076_00353.jpg",
    "1102_MMW_DJI_0076_00361.jpg",
    "1102_MMW_DJI_0076_00369.jpg",
    "1102_MMW_DJI_0076_00377.jpg",
    
    "1102_MMW_DJI_0076_00385.jpg",
    "1102_MMW_DJI_0076_00393.jpg",
    "1102_MMW_DJI_0076_00401.jpg",
    "1102_MMW_DJI_0076_00409.jpg",
    "1102_MMW_DJI_0076_00417.jpg",
    "1102_MMW_DJI_0076_00425.jpg"   
]
test_idx = [18,20,23,26,29,32,39,45,53,54]

In [13]:
image_name[24]

'1102_MMW_DJI_0076_00137.jpg'

In [7]:
import os
path = '/home/cv_stu_3190102787/21_cv_final_project/MMW_BLD/images/'              

for l in range(61):
    name = image_name[l]
    img = path + name           
    image_array=cv2.imread(img)
    
    result_ade = inference_segmentor(model, img)[0]
    result_city = inference_segmentor(model_, img)[0]

    for i in range(image_array.shape[0]):
        for j in range(image_array.shape[1]):
            label_ade = trans(result_ade[i][j])
            label_city = judge(result_city[i][j])
            if label_ade == 7: 
                if i > 600 and j < 600:
                    image_array[i][j]=[7,7,7]
                else:
                    image_array[i][j]=[1,1,1]
            else:
                image_array[i][j]=[label_ade,label_ade,label_ade]
            if label_city == 3:
                image_array[i][j] = [3,3,3]
            if label_city == 2 and label_ade != 3:
                image_array[i][j] = [2,2,2]

            
    path_save = 't_seg/'+str(l)+'.png'
    cv2.imwrite(path_save, image_array, [cv2.IMWRITE_PNG_COMPRESSION, 0])
    print(l)


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
